In [1]:
from gurobipy import *
import random 


import pandas as pd
import numpy as np


"----------------------------------------Read Data------------------------------------------------------------"


# 讀檔案

df_read_csv = pd.read_csv("./10906_final_raw_data/1090603_final_data.csv")
columns_list = ["H/A_ops", "month_ops", "R/L_ops", "TEAM_ops" , "FPCT"]


df = df_read_csv.copy(deep = True).dropna(axis = 0)


# 隊名統一起來


Team_1 = ['中信兄弟', '兄弟']
Team_2 = ['富邦', '義大', '興農']
dict_Team1 = dict.fromkeys(Team_1, '中信兄弟')
dict_Team2 = dict.fromkeys(Team_2, '富邦')
dict_Team3 = {'統一7-ELEVEn' : '統一7-ELEVEn'}
dict_Team_1_2_3 = {**dict_Team1, **dict_Team2, **dict_Team3 }
df['敵隊'] = df['敵隊'].map(dict_Team_1_2_3)


# 正規化 - 計算每個變數的 Z score 後，再用 MinMaxScaler 
for col in columns_list: # 守備率不用正規化
    df[col] = ((df[col] - df[col].mean()) / df[col].std())
    df[col] = ((df[col] - df[col].min()) / (df[col].max() - df[col].min()))
 
    
# 補回原本的 FPCT
df["Raw_FPCT"] = df_read_csv['FPCT']


columns_list = ["H/A_ops", "month_ops", "R/L_ops", "TEAM_ops" , "FPCT", "Raw_FPCT"]
   

# 所有隊伍
teams = list(set(df['敵隊'].values))


# 所有月份
months = [str(m) for m in list(set(df['月'].values))]


# 選擇變數
choose_H_A = int(input('選主/客場(主:1,客:0)\n'))
choose_rival = input('選隊伍('+'\\'.join(teams)+')\n')
choose_month = int(input('選月份('+'\\'.join(months)+')\n'))
choose_pitcher = int(input('選敵方為右投\\左投(右投:0,左投:1)\n'))


# 選擇主場、月份、左/右投、敵隊後的 dataframe，並防呆
while True:

    
    df_choose = df[(df["主場"] == choose_H_A ) & (df["月"]== choose_month )  & (df["左投"]== choose_pitcher) & (df["敵隊"]== choose_rival)]
    
    
    if not df_choose.empty:    
        break
    
    print('\n以前沒發生過 [主場:{} - 敵對:{} - 月份:{} - 右投(0)\\左投(1):{}] 這個對戰組合，請重選。'.format(choose_H_A, choose_rival, choose_month, choose_pitcher))
    # 再重新選擇變數
    choose_H_A = int(input('選主/客場(主:1,客:0)\n'))
    choose_rival = input('選隊伍('+'\\'.join(teams)+')\n')
    choose_month = int(input('選月份('+'\\'.join(months)+')\n'))
    choose_pitcher = int(input('選敵方為右投\\左投(右投:0,左投:1)\n'))


# 計算每位選手平均能力後的 dataframe
df_mean = df_choose.groupby(["球員ID" ]).mean()[columns_list]


# 情境變數 及 代號 [(0, 'H/A_ops'), (1, 'month_ops'), (2, 'R/L_ops'), (3, TEAM_ops), (4, 'FPCT')]
variables_list = list(enumerate(list(df_mean.columns)))


# 選手 及 代號
dict_player = df_mean.reset_index()['球員ID'].to_dict()


# 各球員守過哪裡
df_position = df_read_csv.set_index('球員ID')
df_position1 = df_position['守備位置'].str.replace(")","").str.split("(")
df_position2 = df_position1.apply(lambda x : x[0] if len(x) == 1 else " ".join(x)).reset_index()
df_position3 = df_position2.groupby(['球員ID'])['守備位置'].apply(lambda x : x.unique())


for i in range(len(df_position3)):
    temp = []
    for n in list(df_position3[i]):
        n = n.split()
        temp.extend(n)
    df_position3[i] = set(temp)


# 守備位置代號
position_list = ['C','1B','2B','3B','SS','LF','CF','RF','DH']


dict_position_index = { position_list[i] : i for i in range(len(position_list))}
df_position4 = df_position3.copy(deep = True)


for i in range(len(df_position4)):
    # 刪除投手
    if 'P' in df_position4[i]:
        df_position4[i].remove('P')
    df_position4[i] = [dict_position_index[r] for r in df_position4[i]]


dict_player_position = df_position4.to_dict()
    

row = df_mean.shape[0]
column = df_mean.shape[1] - 1  # 不包含 Raw FPCT

# drop na



x = []
for col in columns_list[:-1] : # 不包含 Raw FPCT
    x.append(df_mean[col].to_list())
x = np.array(x)
x = np.reshape(x, (row, column) )
"----------------------------------------GURTBI------------------------------------------------------------"


# Model
m = Model('Baseball Analysis Project')


# 守備位置代號
position_list = ['C','1B','2B','3B','SS','LF','CF','RF','DH']


# Add parameters
S = x.shape[1] # S個情境變數
P = x.shape[0] # P個球員
R = len(position_list) # R個守備位置



d = {} # 多個守備位置 r1 ~ r9派他上場，為 BINARY，0或1

# x = [] # 每位選手的能力值 ，假設為 0 ~ 1 之 連續正整數
# w = {} # 教練心目中的權重，假設為 0 ~ 100分， w為整數

w = []


for column in columns_list[:-1]:
    temp_w = int(input('請輸入{}權重:'.format(column)))
    w.append(temp_w)
    
    
while sum(w) > 100:
     
        print('\n您輸入的w的總和超過100，請重新輸入')
        w = []
        for column in columns_list[:-1]:
            temp_w = int(input('請輸入{}權重:'.format(column)))
            w.append(temp_w)


for p in range(P):
    for r in range(R):
        player_ID = df_read_csv[df_read_csv["球員ID"] == dict_player[p]]['球員ID'].reset_index(drop = True)[0]
        player_name = df_read_csv[df_read_csv["球員ID"] == dict_player[p]]["先發名單"].reset_index(drop = True)[0]
        d[p,r] = m.addVar(lb = 0, vtype = GRB.BINARY, name = 'd_{}_{}_{}_{}'.format(p, player_name, player_ID, position_list[r]))


# Update
m.update()


# Add objective and constraints
m.setObjective(quicksum(quicksum(quicksum(x[p][s]*w[s]*d[p,r] for r in range(R))for s in range(S)) for p in range(P)),GRB.MAXIMIZE)


for p in range(P):
    m.addConstr(quicksum(d[p,r] for r in range(R)) <= 1 , name='c_第{}球員'.format(p))


for r in range(R):
    m.addConstr(quicksum(d[p,r] for p in range(P)), GRB.EQUAL, 1 , name='c_第{}守備位置'.format(r))

# 守備率設門檻值 

m.addConstr(quicksum(quicksum(df_mean.iloc[p,5]*d[p,r] for r in range(R))for p in range(P))/9 >= df_read_csv['FPCT'].mean() , name='c_第{}守備位置'.format(r))    

# 依照選手實際守備位置，列限制式
for p in range(P): 
    # 沒守過的位置，通通變成 0 
    temp_player_position_list = dict_player_position[dict_player[p]]
    temp_list = set(list(range(len(position_list)))) - set(temp_player_position_list)
    
    m.addConstr(quicksum(d[p,t] for t in temp_list) , GRB.EQUAL, 0 , name='c_第{}球員'.format(p))

#m.addConstr(quicksum(w[s] for s in range(S)), GRB.EQUAL, 100 , name='c_w')

# Result

m.optimize()
#print("選球員:")
player_on_field_list = []
dict_player_position = dict()
print('obj:%d'%m.objVal)
for v in m.getVars():
    if v.x !=0:
        index = v.varName.split("_")[1]
        player_name = v.varName.split("_")[2].replace(" ","").strip()
        player_id = v.varName.split("_")[3]
        player_position = v.varName.split("_")[4]
        
        player_on_field_list.append(player_id)
        dict_player_position[player_id] = player_position
        #print(index, player_name, player_position)
        #print('%s:%d'%(v.varName,v.x))

"--------------------------------------------------------排棒次---------------------------------------------------"
print("排棒次:")
df_on_field = df_mean[df_mean.reset_index()['球員ID'].isin(player_on_field_list).to_list()]
df_on_field_mean = df_on_field.iloc[:,:4].mean(axis = 1)
dict_on_field_mean = df_on_field_mean.to_dict()

sorted_tuples = sorted(dict_on_field_mean.items(), reverse = True ,key = lambda x:x[1])
first_four_tuples = sorted(sorted_tuples[:4],key = lambda x:x[1])
final_order_list = first_four_tuples + sorted_tuples[4:]

for player_id in final_order_list:
    play_name = df_read_csv[df_read_csv["球員ID"] == player_id[0]]["先發名單"].to_list()[0].replace(" ","").strip()
    player_position = dict_player_position[player_id[0]]
    player_FPCT = df_on_field.reset_index()[df_on_field.reset_index()["球員ID"] == player_id[0]]['Raw_FPCT']
    print('[{}]'.format(play_name), '[{}]'.format(player_position) ,'mean of MinMaxScaler(z-score) of ops : [{}]'.format(player_id[1]) , '守備率:{}'.format(player_FPCT.values))



選主/客場(主:1,客:0)
0
選隊伍(統一7-ELEVEn\富邦\中信兄弟)
富邦
選月份(3\4\5\6\7\8\9\10)
5
選敵方為右投\左投(右投:0,左投:1)
0
Using license file C:\Users\Ken Hsieh\gurobi.lic
Academic license - for non-commercial use only
請輸入H/A_ops權重:1
請輸入month_ops權重:1
請輸入R/L_ops權重:1
請輸入TEAM_ops權重:1
請輸入FPCT權重:1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 114 rows, 468 columns and 1696 nonzeros
Model fingerprint: 0xb0fd65d0
Variable types: 0 continuous, 468 integer (468 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 61 rows and 296 columns
Presolve time: 0.04s
Presolved: 53 rows, 172 columns, 499 nonzeros
Variable types: 0 continuous, 172 integer (172 binary)
Found heuristic solution: objective 36.4125530

Root relaxation: objective 3.754702e+01, 78 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D